# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
import xlsxwriter
from scipy import stats

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple Inc',
 'primaryExchange': 'RC) AOQKGNLB AE(SEENGSSTL/ALD TMA',
 'calculationPrice': 'close',
 'open': 135.25,
 'openTime': 1684303182837,
 'openSource': 'iicfloaf',
 'close': 134.3,
 'closeTime': 1662297622066,
 'closeSource': 'fcaofili',
 'high': 131.69,
 'highTime': 1655618202350,
 'highSource': 'ypm1 iecn5 ruedeei latd',
 'low': 131.027,
 'lowTime': 1643395684674,
 'lowSource': 'pculi5d rm de aeee1ityn',
 'latestPrice': 134.1,
 'latestSource': 'Close',
 'latestTime': 'December 17, 2020',
 'latestUpdate': 1635514665904,
 'latestVolume': 98197185,
 'iexRealtimePrice': 131.258,
 'iexRealtimeSize': 1,
 'iexLastUpdated': 1669512456677,
 'delayedPrice': 133.27,
 'delayedPriceTime': 1662464527996,
 'oddLotDelayedPrice': 132.492,
 'oddLotDelayedPriceTime': 1621003984604,
 'extendedPrice': 131.7,
 'extendedChange': -0.35,
 'extendedChangePercent': -0.00278,
 'extendedPriceTime': 1663826632019,
 'previousClose': 129.59,
 'previousVolume': 100749430,
 

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
price = data['latestPrice']
pe_ratio = data['peRatio']
pe_ratio

39.77

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
        pd.Series(
        [
            symbol,
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['peRatio'],
            'N/A'
        ], index=my_columns), ignore_index=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,120.76,55.98,N/A
1,AAL,17.60,-1.2,N/A
2,AAP,166.81,23.48,N/A
3,AAPL,134.80,40.56,N/A
4,ABBV,105.98,22.9,N/A
...,...,...,...,...
500,YUM,111.09,34.16,N/A
501,ZBH,155.83,-214.32,N/A
502,ZBRA,382.64,44.17,N/A
503,ZION,43.19,18.69,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace=True)
final_dataframe.drop('index', axis=1, inplace=True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,33.97,2.02,N/A
1,NLOK,21.21,3.98,N/A
2,AIV,4.74,4.68,N/A
3,UNM,22.18,4.91,N/A
4,BIO,592.62,5.15,N/A
5,KIM,15.67,6.81,N/A
6,MET,47.10,7.72,N/A
7,ALL,111.05,7.82,N/A
8,KR,32.00,8.22,N/A
9,CPB,49.88,8.37,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:10000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for row in final_dataframe.index:
    final_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[row, 'Price'])

final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,NRG,33.97,2.02,5887
1,NLOK,21.21,3.98,9429
2,AIV,4.74,4.68,42194
3,UNM,22.18,4.91,9017
4,BIO,592.62,5.15,337
5,KIM,15.67,6.81,12763
6,MET,47.10,7.72,4246
7,ALL,111.05,7.82,1800
8,KR,32.00,8.22,6250
9,CPB,49.88,8.37,4009


## Building a Better (and More Realistic) Value Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

# P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
EBITA = data[symbol]['advanced-stats']['EBITA']
ev_to_ebita = enterprise_value/EBITA

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit




Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [11]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns=rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
        pd.Series([
            symbol,
            data[symbol]['quote']['latestPrice'],
            'N/A',
            data[symbol]['quote']['peRatio'],
            'N/A',
            data[symbol]['advanced-stats']['priceToBook'],
            'N/A',
            data[symbol]['advanced-stats']['priceToSales'],
            'N/A',
            ev_to_ebitda,
            'N/A',
            ev_to_gross_profit,
            'N/A',
            'N/A'
        ], index=rv_columns), ignore_index = True
        )

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.61,N/A,54.49,N/A,7.48,N/A,7.07,N/A,34.611980,N/A,13.744929,N/A,N/A
1,AAL,17.40,N/A,-1.2,N/A,-1.85,N/A,0.4161,N/A,-5.999628,N/A,1.389375,N/A,N/A
2,AAP,161.96,N/A,24.08,N/A,2.9,N/A,1.1,N/A,10.920613,N/A,2.503679,N/A,N/A
3,AAPL,134.50,N/A,39.61,N/A,34.62,N/A,8.05,N/A,28.476448,N/A,21.300001,N/A,N/A
4,ABBV,107.82,N/A,23.4,N/A,12.17,N/A,4.68,N/A,15.528400,N/A,9.380494,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.48,N/A,32.57,N/A,-4.11,N/A,6.21,N/A,23.527348,N/A,10.245048,N/A,N/A
501,ZBH,151.76,N/A,-207.13,N/A,2.67,N/A,4.58,N/A,25.735534,N/A,7.714189,N/A,N/A
502,ZBRA,383.40,N/A,44.9,N/A,10.68,N/A,4.7,N/A,30.245749,N/A,11.213042,N/A,N/A
503,ZION,42.91,N/A,17.86,N/A,0.9924,N/A,2.34,N/A,10.305511,N/A,2.625339,N/A,N/A


In [12]:
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.61,N/A,54.49,N/A,7.48,N/A,7.07,N/A,34.611980,N/A,13.744929,N/A,N/A
1,AAL,17.40,N/A,-1.2,N/A,-1.85,N/A,0.4161,N/A,-5.999628,N/A,1.389375,N/A,N/A
2,AAP,161.96,N/A,24.08,N/A,2.9,N/A,1.1,N/A,10.920613,N/A,2.503679,N/A,N/A
3,AAPL,134.50,N/A,39.61,N/A,34.62,N/A,8.05,N/A,28.476448,N/A,21.300001,N/A,N/A
4,ABBV,107.82,N/A,23.4,N/A,12.17,N/A,4.68,N/A,15.528400,N/A,9.380494,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.48,N/A,32.57,N/A,-4.11,N/A,6.21,N/A,23.527348,N/A,10.245048,N/A,N/A
501,ZBH,151.76,N/A,-207.13,N/A,2.67,N/A,4.58,N/A,25.735534,N/A,7.714189,N/A,N/A
502,ZBRA,383.40,N/A,44.9,N/A,10.68,N/A,4.7,N/A,30.245749,N/A,11.213042,N/A,N/A
503,ZION,42.91,N/A,17.86,N/A,0.9924,N/A,2.34,N/A,10.305511,N/A,2.625339,N/A,N/A


## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [15]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
40,AON,217.100,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,232.980,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11.000,N/A,9.86,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
136,DISCK,25.340,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
165,ETFC,50.430,N/A,14.86,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
168,EVRG,54.440,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
190,FOX,28.600,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
192,FRC,143.500,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
194,FTI,9.690,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
204,GOOG,1800.800,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [16]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace=True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [17]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [21]:
from scipy.stats import percentileofscore as score

metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for metric in metrics.keys():
    for row in rv_dataframe.index:
        rv_dataframe.loc[row, metrics[metric]] = score(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100
        
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.61,N/A,54.49,0.857426,7.4800,0.726733,7.0700,0.805941,34.611980,0.851485,13.744929,0.819802,N/A
1,AAL,17.40,N/A,-1.20,0.140594,-1.8500,0.0455446,0.4161,0.0316832,-5.999628,0.0277228,1.389375,0.0534653,N/A
2,AAP,161.96,N/A,24.08,0.453465,2.9000,0.4,1.1000,0.159406,10.920613,0.259406,2.503679,0.126733,N/A
3,AAPL,134.50,N/A,39.61,0.730693,34.6200,0.962376,8.0500,0.843564,28.476448,0.80396,21.300001,0.948515,N/A
4,ABBV,107.82,N/A,23.40,0.445545,12.1700,0.847525,4.6800,0.665347,15.528400,0.475248,9.380494,0.637624,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.48,N/A,32.57,0.643564,-4.1100,0.0415842,6.2100,0.770297,23.527348,0.726733,10.245048,0.683168,N/A
501,ZBH,151.76,N/A,-207.13,0.019802,2.6700,0.373267,4.5800,0.619802,25.735534,0.768317,7.714189,0.49901,N/A
502,ZBRA,383.40,N/A,44.90,0.780198,10.6800,0.829703,4.7000,0.669307,30.245749,0.831683,11.213042,0.716832,N/A
503,ZION,42.91,N/A,17.86,0.340594,0.9924,0.0990099,2.3400,0.393069,10.305511,0.229703,2.625339,0.136634,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [23]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentile = []
    for metric in metrics.keys():
        value_percentile.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentile)
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.61,N/A,54.49,0.857426,7.4800,0.726733,7.0700,0.805941,34.611980,0.851485,13.744929,0.819802,0.812277
1,AAL,17.40,N/A,-1.20,0.140594,-1.8500,0.0455446,0.4161,0.0316832,-5.999628,0.0277228,1.389375,0.0534653,0.059802
2,AAP,161.96,N/A,24.08,0.453465,2.9000,0.4,1.1000,0.159406,10.920613,0.259406,2.503679,0.126733,0.279802
3,AAPL,134.50,N/A,39.61,0.730693,34.6200,0.962376,8.0500,0.843564,28.476448,0.80396,21.300001,0.948515,0.857822
4,ABBV,107.82,N/A,23.40,0.445545,12.1700,0.847525,4.6800,0.665347,15.528400,0.475248,9.380494,0.637624,0.614257
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,113.48,N/A,32.57,0.643564,-4.1100,0.0415842,6.2100,0.770297,23.527348,0.726733,10.245048,0.683168,0.573069
501,ZBH,151.76,N/A,-207.13,0.019802,2.6700,0.373267,4.5800,0.619802,25.735534,0.768317,7.714189,0.49901,0.45604
502,ZBRA,383.40,N/A,44.90,0.780198,10.6800,0.829703,4.7000,0.669307,30.245749,0.831683,11.213042,0.716832,0.765545
503,ZION,42.91,N/A,17.86,0.340594,0.9924,0.0990099,2.3400,0.393069,10.305511,0.229703,2.625339,0.136634,0.239802


## Selecting the 50 Best Value Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [26]:
rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop=True, inplace=True)
rv_dataframe

<ipython-input-26-e1a7b46c451c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rv_dataframe.sort_values('RV Score', ascending=True, inplace=True)


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,22.130,N/A,4.88,0.156436,0.4254,0.049505,0.3834,0.029703,2.518919,0.0336634,0.383208,0.0039604,0.0546535
1,AIG,38.770,N/A,-6.92,0.106931,0.5274,0.0574257,0.7214,0.0633663,4.218930,0.039604,0.709684,0.0138614,0.0562376
2,AAL,17.400,N/A,-1.20,0.140594,-1.8500,0.0455446,0.4161,0.0316832,-5.999628,0.0277228,1.389375,0.0534653,0.059802
3,HFC,25.980,N/A,-26.44,0.0673267,0.7833,0.0792079,0.3008,0.0158416,5.949098,0.0574257,2.133738,0.108911,0.0657426
4,MET,47.560,N/A,7.82,0.164356,0.5830,0.0594059,0.6584,0.0574257,4.249258,0.0415842,0.645300,0.0118812,0.0669307
5,F,9.530,N/A,-17.50,0.0811881,1.1800,0.131683,0.2790,0.0118812,4.746510,0.0455446,2.145464,0.112871,0.0766337
6,HIG,47.600,N/A,10.03,0.186139,0.9677,0.0930693,0.8396,0.0871287,4.030724,0.0376238,0.787837,0.019802,0.0847525
7,ALL,106.830,N/A,7.71,0.162376,1.3100,0.164356,0.7275,0.0653465,2.349283,0.0316832,0.711119,0.0158416,0.0879208
8,BA,229.180,N/A,-28.30,0.0653465,-10.9700,0.029703,2.0600,0.350495,-29.725613,0.0158416,-216.417475,0.0019802,0.0926733
9,KSS,40.010,N/A,-58.59,0.049505,1.3000,0.156436,0.3618,0.0257426,9.601044,0.19604,1.452291,0.0574257,0.0970297


## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [27]:
portfolio_input()

Enter the value of your portfolio:100000000


In [29]:
position_size = float(portfolio_size)/len(rv_dataframe.index)

for row in rv_dataframe.index:
    rv_dataframe.loc[row, 'Number of Shares to Buy'] = math.floor(position_size/rv_dataframe.loc[row, 'Price'])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,UNM,22.130,90375,4.88,0.156436,0.4254,0.049505,0.3834,0.029703,2.518919,0.0336634,0.383208,0.0039604,0.0546535
1,AIG,38.770,51586,-6.92,0.106931,0.5274,0.0574257,0.7214,0.0633663,4.218930,0.039604,0.709684,0.0138614,0.0562376
2,AAL,17.400,114942,-1.20,0.140594,-1.8500,0.0455446,0.4161,0.0316832,-5.999628,0.0277228,1.389375,0.0534653,0.059802
3,HFC,25.980,76982,-26.44,0.0673267,0.7833,0.0792079,0.3008,0.0158416,5.949098,0.0574257,2.133738,0.108911,0.0657426
4,MET,47.560,42052,7.82,0.164356,0.5830,0.0594059,0.6584,0.0574257,4.249258,0.0415842,0.645300,0.0118812,0.0669307
5,F,9.530,209863,-17.50,0.0811881,1.1800,0.131683,0.2790,0.0118812,4.746510,0.0455446,2.145464,0.112871,0.0766337
6,HIG,47.600,42016,10.03,0.186139,0.9677,0.0930693,0.8396,0.0871287,4.030724,0.0376238,0.787837,0.019802,0.0847525
7,ALL,106.830,18721,7.71,0.162376,1.3100,0.164356,0.7275,0.0653465,2.349283,0.0316832,0.711119,0.0158416,0.0879208
8,BA,229.180,8726,-28.30,0.0653465,-10.9700,0.029703,2.0600,0.350495,-29.725613,0.0158416,-216.417475,0.0019802,0.0926733
9,KSS,40.010,49987,-58.59,0.049505,1.3000,0.156436,0.3618,0.0257426,9.601044,0.19604,1.452291,0.0574257,0.0970297


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [30]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index=False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Float formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [31]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [32]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Our Excel Output
As before, saving our Excel output is very easy:

In [33]:
writer.save()